In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from text_untils import *
import json

In [2]:
## 1. Get all questions

In [3]:
def getCollection(database, collection):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database]
    cl = db[collection]

    pipeline = [
        {
            "$match": {
                "title": {"$nin": [None, ""]},
            }
        }
    ]
    docs = list(cl.aggregate(pipeline))
    # client.close()
    return docs

In [4]:
documents = getCollection('lawlaboratory', 'temp')

In [5]:
df = pd.DataFrame(documents)
df.sample(5)

,_id,title,date_answer,field,reference,quote,conclusion,description
7357,653a8e291e06875841d16985,Bồi thường về đất đối với những người đang có ...,2019-04-03,Thu hồi đất,NaN,NaN,[Việc bồi thường về đất đối với những người đa...,Tôi muốn hỏi Ban biên tập một câu như sau: Việ...
6935,653a8d701e06875841d16639,Lấy ý kiến về quy hoạch năng lượng nguyên tử,2019-05-30,,NaN,NaN,[Căn cứ Điều 20 Nghị định 41/2019/NĐ-CP quy đị...,"Xin chào, Ban biên tập vui lòng tư vấn giúp tô..."
13999,653a98af1e06875841d19d69,Quy định về nhận ca trực vận hành hệ thống điện,2017-06-27,,NaN,NaN,[Nhận ca trực vận hành hệ thống điện được quy ...,Nhận ca trực vận hành hệ thống điện được quy đ...
11231,653a93741e06875841d187c9,Nhiệm vụ quy hoạch lưu vực sông được quy định ...,2018-03-21,,NaN,NaN,[Nhiệm vụ quy hoạch lưu vực sông được quy định...,Nhiệm vụ quy hoạch lưu vực sông được quy định ...
9010,653a90ca1e06875841d1766f,Hoàn thuế bảo vệ môi trường trong những trường...,2018-11-17,,NaN,NaN,[Tại Điều 8 Thông tư 152/2011/TT-BTC quy định ...,"Chào anh chị, theo tôi được biết thuế bảo vệ m..."


In [6]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20136 entries, 0 to 20135
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          20136 non-null  object
 1   title        20136 non-null  object
 2   date_answer  20136 non-null  object
 3   field        20136 non-null  object
 4   reference    4156 non-null   object
 5   quote        4156 non-null   object
 6   conclusion   20136 non-null  object
 7   description  15980 non-null  object
dtypes: object(8)
memory usage: 1.2+ MB


In [7]:
## 2. Data cleaning

In [8]:
df = df.dropna(subset=['title'])
df = df[df['title'] != '']

In [9]:
def dropEmpty(row):
    if isinstance(row['conclusion'], list) and not row['conclusion'] and pd.isna(row['quote']):
        return True
    return False

df = df[~df.apply(dropEmpty, axis=1)]

df = df.reset_index(drop=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20105 entries, 0 to 20104
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          20105 non-null  object
 1   title        20105 non-null  object
 2   date_answer  20105 non-null  object
 3   field        20105 non-null  object
 4   reference    4156 non-null   object
 5   quote        4156 non-null   object
 6   conclusion   20105 non-null  object
 7   description  15949 non-null  object
dtypes: object(8)
memory usage: 1.2+ MB


In [11]:
# df.to_json('data/backup/temp.json', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)

In [12]:
def merge_quote(quote):
    if isinstance(quote, dict):
        name = quote.get('name', ' ')
        content = '\n'.join(quote.get('content', []))

        if name is None:
            name = ' '
        if content is None:
            content = ' '

        return f"{name}: \n {content}"
    else:
        return None


df['conclusion'] = df.apply(lambda row: merge_quote(row['quote']) if row['conclusion'] == [] else row['conclusion'], axis=1)

In [13]:
df = df.drop(['date_answer', 'field', 'description', 'reference', 'quote'], axis=1)

In [14]:
df.sample(5)

,_id,title,conclusion
6612,653a8cf21e06875841d163c7,05 căn cứ tính tiền cấp quyền khai thác khoáng...,[Theo Điều 5 Nghị định 67/2019/NĐ-CP (Có hiệu ...
5982,653a8c041e06875841d15edb,Tiêu chí đánh giá hạ tầng kỹ thuật công nghệ t...,[Căn cứ Mục 1 Phụ lục 3 Quyết định 2899/QĐ-BTN...
11039,653a93371e06875841d1865d,Nguyên tắc thực hiện khởi động đen và khôi phụ...,[Nguyên tắc thực hiện khởi động đen và khôi ph...
11341,653a93a71e06875841d188b9,Nguyên tắc lập quy hoạch tài nguyên nước,[Theo quy định tại Điều 16 Luật tài nguyên nướ...
19773,653aa03e1e06875841d1caa5,Có được đền bù thiệt hại về đất hay không,"[Chào bạn, Theo nội dung bạn trình bày thì đất..."


In [15]:
def printConclustion(df, target_title):
    conclusion_value = df.loc[df['title'] == target_title, 'conclusion'].values[0]

    if isinstance(conclusion_value, list):
        conclusion_value = '\n'.join(conclusion_value)

    if conclusion_value and isinstance(conclusion_value, str):
        print(f"'{target_title}': {conclusion_value}")
    else:
        print(f"'{target_title}'")

printConclustion(df, "Khi nào phải đăng ký biến động đất đai?")

'Khi nào phải đăng ký biến động đất đai?': Căn cứ theo khoản 4 Điều 95 Luật Đất đai 2013 thì đăng ký biến động được thực hiện đối với trường hợp đã được cấp Giấy chứng nhận hoặc đã đăng ký mà có thay đổi sau đây:
- Người sử dụng đất, chủ sở hữu tài sản gắn liền với đất thực hiện các quyền chuyển đổi, chuyển nhượng, cho thuê, cho thuê lại, thừa kế, tặng cho quyền sử dụng đất, tài sản gắn liền với đất; thế chấp, góp vốn bằng quyền sử dụng đất, tài sản gắn liền với đất;
- Người sử dụng đất, chủ sở hữu tài sản gắn liền với đất được phép đổi tên;
- Có thay đổi về hình dạng, kích thước, diện tích, số hiệu, địa chỉ thửa đất;
- Có thay đổi về tài sản gắn liền với đất so với nội dung đã đăng ký;
- Chuyển mục đích sử dụng đất;
- Có thay đổi thời hạn sử dụng đất;
- Chuyển từ hình thức Nhà nước cho thuê đất thu tiền thuê đất hàng năm sang hình thức thuê đất thu tiền một lần cho cả thời gian thuê; từ hình thức Nhà nước giao đất không thu tiền sử dụng đất sang hình thức thuê đất; từ thuê đất sang gi

In [16]:
def clean_conclusionArray(conclusion):
    words_to_remove = ["tải về", "mẫu đơn", "hình từ internet", "hình ảnh internet", "hình internet", "trân trọng"]
    if isinstance(conclusion, list):
        cleaned_conclusion1 = [word for word in conclusion if isinstance(word, str) and not any(remove_word in word.lower() for remove_word in words_to_remove)]
        cleaned_conclusion2 = [x for x in cleaned_conclusion1 if x is not None and x != '']
        return cleaned_conclusion2
    return conclusion

df['conclusion'] = df['conclusion'].apply(clean_conclusionArray)

In [17]:
def array_to_string(conclusion_array):
    return ' '.join(conclusion_array)

df['conclusion'] = df['conclusion'].apply(array_to_string)

In [18]:
df['title'] = df['title'].apply(clean_text)
df['conclusion'] = df['conclusion'].apply(clean_text)

In [19]:
row_to_save = df.iloc[1]

df_to_save = pd.DataFrame([row_to_save])

df_to_save.to_csv('output.txt', mode='a', index=False, header=None, sep='\n')

In [20]:
df['title'] = df['title'].apply(word_segment)
df['conclusion'] = df['conclusion'].apply(word_segment)

In [21]:
row_to_save = df.iloc[1]
df_to_save = pd.DataFrame([row_to_save])
df_to_save.to_csv('output.txt', mode='a', index=False, header=None, sep='\n')

In [22]:
df['title'] = df['title'].apply(normalize_text)
df['conclusion'] = df['conclusion'].apply(normalize_text)

In [23]:
row_to_save = df.iloc[1]
df_to_save = pd.DataFrame([row_to_save])
df_to_save.to_csv('output.txt', mode='a', index=False, header=None, sep='\n')

In [24]:
df['title'] = df['title'].apply(remove_stopword)
df['conclusion'] = df['conclusion'].apply(remove_stopword)

In [25]:
row_to_save = df.iloc[1]
df_to_save = pd.DataFrame([row_to_save])
df_to_save.to_csv('output.txt', mode='a', index=False, header=None, sep='\n')

In [26]:
# df['title'] = df['title'].apply(strip_answer_string)
# df['conclusion'] = df['conclusion'].apply(strip_answer_string)

In [27]:
df['answer'] = df['conclusion']
df = df.drop('conclusion', axis=1)

In [28]:
df['title'] = df['title'].str.replace('_', ' ')
df['answer'] = df['answer'].str.replace('_', ' ')

In [29]:
df = df.drop_duplicates(subset='title', keep = 'first')
df.duplicated(subset='title').sum()

0

In [30]:
df.to_json('data/cleaned/temp.jsonl', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)

In [31]:
json_objects = []

with open('./data/backup/temp.json', 'r', encoding='utf-8') as file:
    for line in file:
        # Load each line as a JSON object
        try:
            json_object = json.loads(line)
            json_objects.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

df_backup = pd.json_normalize(json_objects)

In [32]:
df_backup.sample(5)

,_id,title,date_answer,field,reference,conclusion,description,quote.content,quote.name
18645,653a9ebc1e06875841d1c253,"Tính toán, xác định các khoảng cách phục vụ xá...",2016-11-12,,None,[Theo quy định hiện hành tại Thông tư 29/2016/...,"Tính toán, xác định các khoảng cách phục vụ xá...",None,None
5995,653a8c231e06875841d15f4b,Điểm tiêu đo là gì?,2019-11-22,,None,[Điểm tiêu đo được quy định tại Khoản 4 Điều 3...,Ban biên tập cho tôi hỏi: Theo quy định từ Bộ ...,None,None
16674,653a9b321e06875841d1b2df,Trách nhiệm của cơ quan Cảnh sát phòng cháy và...,2017-02-17,,None,[Theo quy định tại Khoản 6 Điều 16 Nghị định 7...,Trách nhiệm của cơ quan Cảnh sát phòng cháy và...,None,None
3530,653a87e31e06875841d14bd5,Phải làm Kiểm soát viên chính thị trường bao n...,2022-05-19,,Theo Khoản 5 Điều 5 Thông tư 02/2022/TT-BCT qu...,[Theo Khoản 5 Điều 5 Thông tư 02/2022/TT-BCT q...,None,[],None
8639,653a90411e06875841d17403,Điều kiện tiến hành phiên họp chính thức của h...,2018-12-22,Đánh giá môi trường chiến lược,None,[Điều kiện tiến hành phiên họp chính thức của ...,Ban biên tập có nhận được thăc mắc của bạn Ngu...,None,None


In [33]:
df_backup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20022 entries, 0 to 20021
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            20022 non-null  object
 1   title          20022 non-null  object
 2   date_answer    20022 non-null  object
 3   field          20022 non-null  object
 4   reference      4119 non-null   object
 5   conclusion     20022 non-null  object
 6   description    15903 non-null  object
 7   quote.content  4119 non-null   object
 8   quote.name     1820 non-null   object
dtypes: object(9)
memory usage: 1.4+ MB


In [34]:
def clean_conclusionArray(conclusion):
    words_to_remove = ["tải về", "mẫu đơn", "hình từ internet", "hình ảnh internet", "hình internet", "trân trọng"]
    if isinstance(conclusion, list):
        cleaned_conclusion1 = [word for word in conclusion if isinstance(word, str) and not any(remove_word in word.lower() for remove_word in words_to_remove)]
        cleaned_conclusion2 = [x for x in cleaned_conclusion1 if x is not None and x != '']
        return cleaned_conclusion2
    return conclusion

df_backup['conclusion'] = df_backup['conclusion'].apply(clean_conclusionArray)

In [36]:
# df_backup = df_backup.drop(['quote'], axis=1)

In [37]:
df_backup.to_json('data/backup/temp.json', orient='records', lines=True, force_ascii=False, date_format='iso', default_handler=str)